In [1]:
import pandas as pd
file_name = 'data/qa_queries_V28.parquet'
#load data
df = pd.read_parquet(file_name)

In [2]:
# sort by lat, time
df = df.sort_values(by=['lat', 'time'])

In [3]:
df.columns

Index(['point', 'time', 'lat', 'lon', 'ndvi.landsat', 'qa.landsat',
       'ndvi.modis', 'qa.modis', 'ndvi.sentinel2', 'qa.sentinel2',
       'ndvi.streambatch', 'source.streambatch'],
      dtype='object')

In [4]:
# slice off all rows where qa.sentinel2 == 1
s2 = df[df['qa.sentinel2'] == 1].copy()

# remove all columns except time, lat, ndvi.sentinel2
s2 = s2[['time', 'lat', 'ndvi.sentinel2']]
# rename ndvi.sentinel2 to ndvi
s2 = s2.rename(columns={'ndvi.sentinel2': 'ndvi'})
# remove any row where ndvi is the same as the previous row
s2 = s2[s2['ndvi'] != s2['ndvi'].shift(1)]

# slice off all rows where qa.landsat8 == 1
l8 = df[df['qa.landsat'] == 1].copy()
# remove all columns except time, lat, ndvi.landsat
l8 = l8[['time', 'lat', 'ndvi.landsat']]
# rename ndvi.landsat to ndvi
l8 = l8.rename(columns={'ndvi.landsat': 'ndvi'})
# remove any row where ndvi is the same as the previous row
l8 = l8[l8['ndvi'] != l8['ndvi'].shift(1)]
# concat s2 and l8
m = pd.concat([s2, l8])
# sort by lat, time
m = m.sort_values(by=['lat', 'time'])
# reset index
m = m.reset_index(drop=True)
m.head()


,time,lat,ndvi
0,2013-04-22,-34.065146,0.451291
1,2013-07-20,-34.065146,0.718348
2,2013-08-05,-34.065146,0.688542
3,2013-08-12,-34.065146,0.763199
4,2013-09-06,-34.065146,0.680351


In [5]:
# get all the unique lats
lats = df['lat'].unique()

In [10]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from pykalman import KalmanFilter


def f(m):

    # Step 2: Create a new DataFrame 'm1' with all dates from the minimum to maximum date in 'm'
    min_date = m["time"].min()
    max_date = m["time"].max()
    date_range = pd.date_range(min_date, max_date, freq='D')  # Create a date range with daily frequency
    m1 = pd.DataFrame({"time": date_range})

    # Step 3: Merge the original 'ndvi' values into the new DataFrame 'm1' using outer join
    m1 = pd.merge(m1, m, on="time", how="left")

    # Step 2: Define Kalman filter model parameters
    # We assume a simple constant velocity model for this example
    transition_matrix = [[1, 1], [0, 1]]  # State transition matrix
    observation_matrix = np.array([[1, 0]])  # Observation matrix
    initial_state_mean = [m1["ndvi"].dropna().iloc[0], 0]  # Initial state [initial value, initial velocity]
    initial_state_covariance = np.eye(2)  # Initial covariance matrix
    observation_covariance = np.eye(1)  # Observation covariance matrix
    transition_covariance = np.eye(2)  # State transition covariance matrix

    # Step 3: Implement Kalman filter algorithm to estimate missing values
    # Initialize the Kalman filter with the model parameters
    kf = KalmanFilter(
        transition_matrices=transition_matrix,
        observation_matrices=observation_matrix,
        initial_state_mean=initial_state_mean,
        initial_state_covariance=initial_state_covariance,
        observation_covariance=observation_covariance,
        transition_covariance=transition_covariance
    )

    # Store observed "ndvi" values (without NaNs) in an array
    observed_ndvi = m1["ndvi"].dropna().values.reshape(-1, 1)

    # Use the Kalman filter to estimate the missing values (NaNs)
    (filtered_state_means, _) = kf.filter(observed_ndvi)

    # Create a new DataFrame with the estimated "ndvi" values
    m1_estimated = m1.copy()
    m1_estimated.loc[m1["ndvi"].isnull(), "ndvi"] = filtered_state_means[-m1["ndvi"].isnull().sum():]



    return m1


In [11]:
start_date = '2020-01-01'
end_date = '2022-12-31'
m1 = m[m['lat'] == lats[0]]
m1 = m1[(m1['time'] >= start_date) & (m1['time'] <= end_date)]
fitted = f(m1)
# fitted['ndvi_'] = fitted['ndvi'].interpolate(method='linear', limit_direction='both')
fitted.head()

ValueError: Must have equal len keys and value when setting with an ndarray

In [ ]:
import matplotlib.pyplot as plt
start_date = '2020-01-01'
end_date = '2022-12-31'
for l in lats:
    this = df[df['lat'] == l]
    m1 = m[m['lat'] == l]
    this = this[(this['time'] >= start_date) & (this['time'] <= end_date)]
    m1 = m1[(m1['time'] >= start_date) & (m1['time'] <= end_date)]

    
    fitted = f(m1)


    # plot ndvi.streambatch vs time
    plt.scatter(this['time'], this['ndvi.streambatch'], label='streambatch',s=5,color='orange')
    plt.scatter(fitted['time'], fitted['ndvi_smoothed'], label='savgol',s=5,color='blue')
    plt.scatter(m1['time'], m1['ndvi'], label='sentinel and landsat',s=14, color="red")
    plt.title(f"lat: {l} plot 1: Streambatch plot2: savgol applied to just s2 and landsat")
    plt.legend()
    plt.show()

    
    
    # plt.scatter(m1['time'], m1['ndvi'], label='sentinel and landsat',s=10, color="red")
    # plt.scatter(fitted['time'], fitted['ndvi_smoothed'], label='savgol',s=5, color='blue')
    # plt.title(f"lat: {l} same location as above. Sentinel2/Landsat and savgol applied to it")
    # plt.legend()

    # plt.show()

    plt.scatter(fitted['time'], fitted['ndvi_smoothed'], label='savgol',s=5, color='blue')
    plt.title(f"lat: {l} Just Savgol")
    plt.legend()

    plt.show()


    

